- 결과 데이터가 3가지 이상으로 구성되어 있다.
- 다중 분류
- 손실함수 : categorical_crossentropy
- 출력층의 활성화 함수 : softmax
- 출력층의 노드의 개수 : 결과 데이터의 종류 수

In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False
# 매직명령어 => 쥬피터노트북에서 그래프 삽입 기능 
%matplotlib inline
# 글꼴 선명화 
%config InlineBackend.figure_format = 'retina'

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
# 신경망 모델을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형 회귀 레이어
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 객체
from tensorflow.keras.layers import Activation
# 원핫 인코딩을 수행하는 함수
from tensorflow.keras.utils import to_categorical

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
# 문자열 => 숫자
from sklearn.preprocessing import LabelEncoder

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [2]:
# 데이터를 읽어온다.
df1 = pd.read_csv('./data/iris.csv', header=None)
df1.columns = ['꽃받침길이', '꽃받침너비', '꽃잎길이', '꽃잎너비', '품종']
df1.head()

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비,품종
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# 전반적인 정보를 확인한다.
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   꽃받침길이   150 non-null    float64
 1   꽃받침너비   150 non-null    float64
 2   꽃잎길이    150 non-null    float64
 3   꽃잎너비    150 non-null    float64
 4   품종      150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
# 문자열 데이터를 숫자로 변환한다.
encoder1 = LabelEncoder()
df1['품종'] = encoder1.fit_transform(df1['품종'])
df1.head()

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비,품종
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
# 결측치 확인
df1.isna().sum()

꽃받침길이    0
꽃받침너비    0
꽃잎길이     0
꽃잎너비     0
품종       0
dtype: int64

In [6]:
# 입력과 결과로 나눈다.
X = df1.drop('품종', axis=1)
y = df1['품종']

In [7]:
# 2진 분류 옵션들
# out_nodes = 1
# loss_function = 'binary_crossentropy'
# activation_function = 'sigmoid'

# 다중 분류 옵션들
out_nodes = len(y.value_counts())
loss_function = 'categorical_crossentropy'
activation_function = 'softmax'

In [8]:
# 표준화
scaler1 = StandardScaler()
X = scaler1.fit_transform(X)
X

array([[-9.00681170e-01,  1.03205722e+00, -1.34127240e+00,
        -1.31297673e+00],
       [-1.14301691e+00, -1.24957601e-01, -1.34127240e+00,
        -1.31297673e+00],
       [-1.38535265e+00,  3.37848329e-01, -1.39813811e+00,
        -1.31297673e+00],
       [-1.50652052e+00,  1.06445364e-01, -1.28440670e+00,
        -1.31297673e+00],
       [-1.02184904e+00,  1.26346019e+00, -1.34127240e+00,
        -1.31297673e+00],
       [-5.37177559e-01,  1.95766909e+00, -1.17067529e+00,
        -1.05003079e+00],
       [-1.50652052e+00,  8.00654259e-01, -1.34127240e+00,
        -1.18150376e+00],
       [-1.02184904e+00,  8.00654259e-01, -1.28440670e+00,
        -1.31297673e+00],
       [-1.74885626e+00, -3.56360566e-01, -1.34127240e+00,
        -1.31297673e+00],
       [-1.14301691e+00,  1.06445364e-01, -1.28440670e+00,
        -1.44444970e+00],
       [-5.37177559e-01,  1.49486315e+00, -1.28440670e+00,
        -1.31297673e+00],
       [-1.26418478e+00,  8.00654259e-01, -1.22754100e+00,
      

In [9]:
# 원 핫 인코딩
# 결과데이터가 0, 1, 2라면...
# 0 => 1, 0, 0
# 1 => 0, 1, 0
# 2 => 0, 0, 1
# 로 환산하여 학습을 한다.
# 예측된 결과는 가장 값이 큰 자리의 순서 값으로 결정한다.
y = to_categorical(y)
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [10]:
# 입력노드의 개수
# 입력 데이터 행 하나의 컬럼의 개수
input_size = X.shape[1]
input_size

4

In [11]:
# 신경망 설계
model = Sequential()

model.add(Dense(12, input_dim=input_size))
model.add(Activation('relu'))

model.add(Dense(out_nodes))
model.add(Activation(activation_function))

In [12]:
# 모델 컴파일
model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                60        
                                                                 
 activation (Activation)     (None, 12)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 39        
                                                                 
 activation_1 (Activation)   (None, 3)                 0         
                                                                 
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 학습
# verbose에 0을 넣어주면 학습시 출력을 수행하지 않아서
# 약간의 이득을 볼 수 있다.
# model.fit(X, y, epochs=200, batch_size=10, verbose=0)
model.fit(X, y, epochs=200, batch_size=10)

Epoch 1/200
15/15 [==============================] - 1s 2ms/step - loss: 1.2753 - accuracy: 0.5400
Epoch 2/200
15/15 [==============================] - 0s 2ms/step - loss: 1.1812 - accuracy: 0.5667
Epoch 3/200
15/15 [==============================] - 0s 2ms/step - loss: 1.0970 - accuracy: 0.6000
Epoch 4/200
15/15 [==============================] - 0s 3ms/step - loss: 1.0105 - accuracy: 0.6267
Epoch 5/200
15/15 [==============================] - 0s 2ms/step - loss: 0.9361 - accuracy: 0.6267
Epoch 6/200
15/15 [==============================] - 0s 2ms/step - loss: 0.8659 - accuracy: 0.6400
Epoch 7/200
15/15 [==============================] - 0s 2ms/step - loss: 0.8035 - accuracy: 0.6133
Epoch 8/200
15/15 [==============================] - 0s 2ms/step - loss: 0.7419 - accuracy: 0.6133
Epoch 9/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6898 - accuracy: 0.7267
Epoch 10/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 0.7600
Epoch 11/

In [14]:
# 정확도 확인
a1 = model.evaluate(X, y)
print(f'손실률 : {a1[0]}')
print(f'정확도 : {a1[1]}')

5/5 [==============================] - 0s 2ms/step - loss: 0.0628 - accuracy: 0.9733
손실률 : 0.06278061121702194
정확도 : 0.9733333587646484
